## Wood's方程

In [31]:
import math

# water
vp_water = 1490
rho_water = 1
phi_water = 0.3

# Quartz
phi_quartz = 0.8 * (1 - phi_water)
vp_quartz = 6008.4
rho_quartz = 2.65
mu_quartz = 44

# Feldspar
phi_Feldspar = 0.2 * (1 - phi_water)
K_Feldspar = 75
rho_Feldspar = 2.63
mu_Feldspar = 26


# Calcalate K_water
K_water = rho_water * vp_water**2
K_water = K_water / 10**6
print("K_water: ", K_water, "GPa")

# Calculate K_quartz
vs_quartz = math.sqrt(mu_quartz / rho_quartz)
print("vs_quartz: ", vs_quartz, "km/s")
K_quartz = rho_quartz * vp_quartz**2 - 4/3 * rho_quartz * vs_quartz

K_quartz = K_quartz / 10**6
print("K_quartz: ", K_quartz, "GPa")

K = []
K.append(K_quartz)
K.append(K_Feldspar)
K.append(K_water)

phi = []
phi.append(phi_quartz)
phi.append(phi_Feldspar)
phi.append(phi_water)

# Calculate average K
res_K = 0
for ki, phii in zip(K, phi):
    res_K = res_K + phii / ki

res_K = 1 / res_K


print("K: ", res_K, "GPa")

# Calculate average rho 
Rho = []
Rho.append(rho_quartz)
Rho.append(rho_Feldspar)

rho = 0
for phii, rhoi in zip(phi, Rho):
    rho = rho + phii * rhoi

print("rho: ", rho, "g/cm^3")

# Calculate wood's Velocity
Velocity_wood = math.sqrt(res_K / rho)

print("wood's P wave Velocity: ", Velocity_wood, "km/s")

K_water:  2.2201 GPa
vs_quartz:  4.074772826171499 km/s
K_quartz:  95.66729258646934 GPa
K:  7.0003825987688355 GPa
rho:  1.8521999999999998 g/cm^3
wood's P wave Velocity:  1.9440926053242045 km/s


## Wyllie时间平均方程

In [32]:
vp_Feldspar = math.sqrt((K_Feldspar + 4/3 * mu_Feldspar) / rho_Feldspar)
print("vp_Feldspar: ", vp_Feldspar, "km/s")
vp_quartz = vp_quartz / 1000
print("vp_quartz: ", vp_quartz, "km/s")

phi = 0.3
V_ma = 0.8 * vp_quartz + 0.2 * vp_Feldspar

print("V_ma: ", V_ma, "km/s")
V_fl = 1.49

V_rock = (1 - phi) / V_ma + phi / V_fl
V_rock = 1 / V_rock

print("V_rock: ", V_rock, "km/s")

vp_Feldspar:  6.457426139317443 km/s
vp_quartz:  6.0084 km/s
V_ma:  6.098205227863489 km/s
V_rock:  3.1632540847908897 km/s


In [34]:
phi = 0.3
varphi1 = 0.8
varphi2 = 0.2
rho_water = 1

rho = (1 - phi) * (varphi1 * rho_quartz + varphi2 * rho_Feldspar) + phi * rho_water
print("average rho: ", rho, "g/cm^3")

# Solve for V_P
vp = (rho / 0.31)**(1/0.25)

vp = vp / 1000

print("V_P:", vp, "km/s")

average rho:  2.1521999999999997 g/cm^3
V_P: 2.323184668133828 km/s


In [36]:
phi = 0.4

sum_K = (1 - phi) / K_quartz + phi / K_water
sum_K = 1 / sum_K

rho = (1 - phi) * rho_quartz + phi * rho_water

V = math.sqrt(sum_K / rho)

print("V_P: ", V, "km/s")

V_P:  1.6417215104007534 km/s


In [56]:
phi = 0.1

# shiying
K_shiying = 37
mu_shiying = 44
rho_shiying = 2650 / 1000

# fangjieshi
K_fang = 76.8
mu_fang = 32
rho_fang = 2710 / 1000

# rho
rho = (1 - phi) * rho_shiying + phi * rho_fang
print("rho: ", rho)

# Voigt
Kv = K_shiying * (1 - phi) + K_fang * phi
print("Kv: ", Kv)

muv = mu_shiying * (1 - phi) + mu_fang * phi
print("muv: ", muv)

Vp_voigt = math.sqrt((Kv + 4/3*muv) / rho)
print("Vp_voigt: ", Vp_voigt)

Vs_voigt = math.sqrt(muv / rho)
print("Vs_voigt: ", Vs_voigt)

# Reuss
Kr = (1 - phi) / K_shiying + phi / K_fang
Kr = 1 / Kr
print("Kr: ", Kr)

mur = (1 - phi) / mu_shiying + phi / mu_fang
mur = 1 / mur
print("mur: ", mur)

Vp_reuss = math.sqrt((Kr + 4/3*mur) / rho)
print("Vp_reuss: ", Vp_reuss)

Vs_reuss = math.sqrt(mur / rho)
print("Vs_reuss: ", Vs_reuss)

rho:  2.6559999999999997
Kv:  40.980000000000004
muv:  42.800000000000004
Vp_voigt:  6.075784775859847
Vs_voigt:  4.014281732928732
Kr:  39.02224663553969
mur:  42.40963855421687
Vp_reuss:  5.998507324821699
Vs_reuss:  3.9959334823559036


In [60]:
def hashin_shtrikman_upper_bound(phi, K1, mu1, K2, mu2):
    K_bnd = K1 + phi / (1 / (K2 - K1) + phi / (K1 + 4/3 * mu1))
    mu_bnd = mu1 + phi / (1 / (mu2 - mu1) + (2 * phi * (K1 + 2 * mu1)) / (5 * mu1 * (K1 + 4/3 * mu1)))
    return K_bnd, mu_bnd

def hashin_shtrikman_lower_bound(phi, K1, mu1, K2, mu2):
    K_bnd = K1 + phi / (1 / (K2 - K1) + phi / (K1 + 4/3 * mu1))
    mu_bnd = mu1 + phi / (1 / (mu2 - mu1) + (2 * phi * (K1 + 2 * mu1)) / (5 * mu1 * (K1 + 4/3 * mu1)))
    return K_bnd, mu_bnd


K1_rock = 76.8e9  # 方解石体积模量，Pa
mu1_rock = 32e9   # 方解石剪切模量，Pa
rho_rock = 2710   # 方解石密度，kg/m^3

K1_quartz = 37e9  # 石英体积模量，Pa
mu1_quartz = 44e9 # 石英剪切模量，Pa
rho_quartz = 2650 # 石英密度，kg/m^3

# 方解石含量为10%
phi = 0.1

# 计算Hashin-Shtrikman上、下界
K_upper, mu_upper = hashin_shtrikman_upper_bound(phi, K1_rock, mu1_rock, K1_quartz, mu1_quartz)
# K_lower, mu_lower = hashin_shtrikman_lower_bound(phi, K1_rock, mu1_rock, K1_quartz, mu1_quartz)
K_lower, mu_lower = hashin_shtrikman_lower_bound(phi, K1_quartz, mu1_rock, K1_rock, mu1_quartz)

print("Hashin-Shtrikman上界：")
print("体积模量 (K_bnd) =", K_upper / 1e9, "GPa")
print("剪切模量 (mu_bnd) =", mu_upper / 1e9, "GPa")

print("\nHashin-Shtrikman下界：")
print("体积模量 (K_bnd) =", K_lower / 1e9, "GPa")
print("剪切模量 (mu_bnd) =", mu_lower / 1e9, "GPa")

Vp_up = math.sqrt((K_upper/1e9 + 4/3*mu_upper/1e9) / rho)
print("Vp_up: ", Vp_up)

Vs_up = math.sqrt(mu_upper/1e9 / rho)
print("Vs_up: ", Vs_up)

Vp_down = math.sqrt((K_lower/1e9 + 4/3*mu_lower/1e9) / rho)
print("Vp_down: ", Vp_down)

Vs_down = math.sqrt(mu_lower/1e9 / rho)
print("Vs_up: ", Vs_down)


Hashin-Shtrikman上界：
体积模量 (K_bnd) = 72.68283784563874 GPa
剪切模量 (mu_bnd) = 33.17915423758554 GPa

Hashin-Shtrikman下界：
体积模量 (K_bnd) = 40.79062724157169 GPa
剪切模量 (mu_bnd) = 33.177605781272455 GPa
Vp_up:  6.634887205054997
Vs_up:  3.5344237783370116
